# Обязательная часть

Будем осуществлять работу с непростым набором данных о состоянии здоровья лошадей, испытывающих кишечные колики.

# Задание 1. Базовое изучение

Изучить представленный набор данных на основе описания его столбцов и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

In [2]:
import pandas as pd

In [57]:
import numpy as np

In [262]:
df = pd.read_csv('horse_data.csv', header=None)

In [263]:
#сначала оставляем только выбранные столбцы

# 0 столбец — была ли операция
          #1 = Да, была операция
          #2 = Лечение прошло без хирургического вмешательства
        
# 1 столбец — возраст
          #1 = >6 месяцев
          #2 = <6 месяцев

# 4 столбец — ректальная температура в градусах Цельсия.
     
# 9 столбец — признак "capillary refill time"
        #1 = <3 секунд
        #2 = =>3 секунд
        
# 11 столбец — перистальтика кишечника лошади. 
        #1 = гипермобильность
        #2 = нормальный
        #3 = замедленная работа кишечника
        #4 = отсутствует
        
# 12 столбец — вздутие живота
        #1 = отсутствует
        #2 = легкое
        #3 = умеренное
        #4 = серьезное
        
# 17 столбец — живот
        #1 = нормальный
        #2 = другое
        #3 = твердый кал в толстой кишке
        #4 = растянутый тонкий кишечник
        #5 = растянутый толстый кишечник

# 22 столбец —  - что в итоге случилось с лошадью?
        #1 = жил
        #2 = умер
        #3 = был усыплен

df.drop(df.columns[[2, 3, 5, 6, 7, 8, 10, 13, 14, 15, 16, 18, 19, 20, 21, 23, 24, 25, 26, 27]], axis=1, inplace=True)

df

,0,1,4,9,11,12,17,22
0,2,1,66,2,4,4,5,2
1,1,1,88,1,4,2,2,3
2,2,1,40,1,3,1,1,1
3,1,9,164,2,4,4,?,2
4,2,1,104,2,?,?,?,2
...,...,...,...,...,...,...,...,...
295,1,1,120,2,4,?,5,3
296,2,1,72,2,3,3,4,3
297,1,1,72,1,4,3,5,2
298,1,1,100,1,3,3,4,1


In [264]:
#в 0 столбце только два значения — была ли операция(1) или не было (2)
#поэтому посчитаем только сколько было операций, а остальные метрики тут считать смысла нет.

df[0].value_counts()

#в большем количестве случаев операция все таки требовалась.

1    180
2    119
?      1
Name: 0, dtype: int64

In [265]:
#в 1 столбце только два значения — взрослая лошадь (1) и жеребенок (2)

df[1].value_counts()

#я заметила ,что в столбце нет значения "2", зато есть "9" и нет никаких других.
#полагаю, что при заполнении данных произошла ошибка и цифра "2" заменилась на "9". Не похоже на опечатку. 
#буду считать, что в исследовании участвовало 24 жеребенка.
#и то, что взрослые лошади чаще переживают проблемы со здоровьем кажется тоже закономерным

1    276
9     24
Name: 1, dtype: int64

In [266]:
#в 4 столбце хранятся данные о ректальной температуре в Цельсиях.

#с помощью df[4].value_counts() посмотрела какие там вообще есть значения.
#обнаружила, что в 24 случаях температура неизвестна, поле заполнено как "?"
#заменила "?" на Nan, чтоб дальше работать с данными
#а затем все столбцы перевела в цисловой вид

df_new = df.replace(r'\D', np.nan, regex=True)

df_new = df_new.apply(lambda column: pd.to_numeric(column), axis=0)

In [267]:
#ну и теперь статистики по ректальной температуре.

print(f'максимум {df_new[4].max()}')
print(f'минимум {df_new[4].min()}')
print(f'среднее арифмитическое {df_new[4].mean()}')
print(f'мода {df_new[4].mode()[0]}')
print(f'медиана {df_new[4].median()}')
print(f'СКО {np.std(df_new[4])}')
print(f'дисперсия {np.var(df_new[4])}')

Q1 = df_new[4].quantile(0.25)
Q3 = df_new[4].quantile(0.75)
IQR = Q3 - Q1
print(f'межквартильный размах {IQR}')

максимум 184.0
минимум 30.0
среднее арифмитическое 71.91304347826087
мода 48.0
медиана 64.0
СКО 28.578642644025653
дисперсия 816.7388153749215
межквартильный размах 40.0


In [ ]:
#нормальная температура 37,8
#судя по моде 48 с данными по температуре что-то не так, температур не может быть выше 42 даже при инфекции.
#считаю, что ректальную температуру нельзя учитывать в исследовании.

In [268]:
#в 9 столбце (признак "capillary refill time") только 2 значения — <3 секунд(1) и =>3 секунд(2)
#посмотрим что встречается чаще

df_new[9].value_counts()

#выское знаниение говорит о полохом кровообращении, но оно наблюдается только у четверти животных.

1.0    188
2.0     78
3.0      2
Name: 9, dtype: int64

In [269]:
# 11 столбец — перистальтика кишечника лошади. 
        #1 = гипермобильность
        #2 = нормальный
        #3 = замедленная работа кишечника
        #4 = отсутствует
        
df_new[11].value_counts()

3.0    128
4.0     73
1.0     39
2.0     16
Name: 11, dtype: int64

In [270]:
# 12 столбец — вздутие живота
        #1 = отсутствует
        #2 = легкое
        #3 = умеренное
        #4 = серьезное
        
df_new[12].value_counts()       

1.0    76
3.0    65
2.0    65
4.0    38
Name: 12, dtype: int64

In [271]:
# 17 столбец — живот
        #1 = нормальный
        #2 = другое
        #3 = твердый кал в толстой кишке
        #4 = растянутый тонкий кишечник
        #5 = растянутый толстый кишечник
        
df_new[17].value_counts()       

5.0    79
4.0    43
1.0    28
2.0    19
3.0    13
Name: 17, dtype: int64

In [272]:
# 22 столбец — что в итоге случилось с лошадью?
        #1 = жил
        #2 = умер
        #3 = был усыплен
        
df_new[22].value_counts()

#больше половины лошадей выжили — это хорошо :)
#121 лошадь все таки умерла — это плохо :(

#мы можем сказать что процент смертности при коликах — 25,7%
#процен усыления — 14,7%

1.0    178
2.0     77
3.0     44
Name: 22, dtype: int64

# Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

In [273]:
#для выбранных мною столбцов есть смысл считать выбросы только по ректальной температуре (столбец 4).
#но я уже писала выше, что считаю, что эти данные не следует учитывать в исследовании.

#вот количество измерений, где температура выше 42 градусов:
df_new[4].gt(42).sum()

242

In [ ]:
#так как при температуре выше 42 лошадь неминуемо погибнет, у нас должно в конце быть минимум 242 мертвые лошади из 300.
#но у нас 178 живых лошадей. так что с температурой что-то не то.

In [274]:
#по столбцу 9  (признак "capillary refill time") я заметила что присутствует значение "3" в двух строках 
#а такого значения быть не может

#видно здесь:
df_new[9].value_counts()

1.0    188
2.0     78
3.0      2
Name: 9, dtype: int64

In [ ]:
#так как таких строк мало, их можно просто не учитывать

In [ ]:
#по остальным строкам я выбросов не заметила никаких

# Задание 3. Работа с пропусками

Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

In [275]:
df_new.head(20)

,0,1,4,9,11,12,17,22
0,2.0,1,66.0,2.0,4.0,4.0,5.0,2.0
1,1.0,1,88.0,1.0,4.0,2.0,2.0,3.0
2,2.0,1,40.0,1.0,3.0,1.0,1.0,1.0
3,1.0,9,164.0,2.0,4.0,4.0,NaN,2.0
4,2.0,1,104.0,2.0,NaN,NaN,NaN,2.0
5,2.0,1,NaN,1.0,3.0,2.0,3.0,1.0
6,1.0,1,48.0,1.0,3.0,3.0,5.0,1.0
7,1.0,1,60.0,1.0,4.0,2.0,4.0,2.0
8,2.0,1,80.0,1.0,4.0,4.0,5.0,3.0
9,2.0,9,90.0,1.0,3.0,1.0,NaN,1.0


In [276]:
#считаем количество пропусков в каждом столбце

pd.isnull(df_new).sum()

0       1
1       0
4      24
9      32
11     44
12     56
17    118
22      1
dtype: int64

In [277]:
# По 0 столбцу (была ли операция) — 1 пропуск. 
#в строчке:

np.where(pd.isnull(df_new[0]))

(array([132], dtype=int64),)

In [182]:
#вот она
df_new.loc[[132]]

,0,1,4,9,11,12,17,22
132,NaN,1,48.0,1.0,3.0,1.0,5.0,NaN


In [278]:
#просто удалю ее.
df_new = df_new.drop(index=[132], axis=1)

df_new

,0,1,4,9,11,12,17,22
0,2.0,1,66.0,2.0,4.0,4.0,5.0,2.0
1,1.0,1,88.0,1.0,4.0,2.0,2.0,3.0
2,2.0,1,40.0,1.0,3.0,1.0,1.0,1.0
3,1.0,9,164.0,2.0,4.0,4.0,NaN,2.0
4,2.0,1,104.0,2.0,NaN,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...
295,1.0,1,120.0,2.0,4.0,NaN,5.0,3.0
296,2.0,1,72.0,2.0,3.0,3.0,4.0,3.0
297,1.0,1,72.0,1.0,4.0,3.0,5.0,2.0
298,1.0,1,100.0,1.0,3.0,3.0,4.0,1.0


In [ ]:
# По 1 столбцу (возраст) — нет пропусков. 

In [279]:
# По 4 столбцу (температура) — 24 пропуска.

#Если бы температура не вызывала бы у меня подозрений, пропуски бы заполнила бы модой 
#самым часто встречающимся значением у больного животного. 

np.where(pd.isnull(df_new[4]))
df_new[4].fillna(df_new[4].mode()[0], inplace=True)

In [210]:
# по 9 столбцу признак "capillary refill time" - 32 пропуска.

In [211]:
# по 11 столбцу (перистальтика кишечника) — 44 пропуска. 

In [194]:
# по 12 столбцу (вздутие живота) — 56 пропусков.

In [193]:
# по 17 столбцу (живот) — 118 пропусков.

In [285]:
#вот эти столбцы: 9,11,12,17 не хочу заполнять модой, там нет "супер типичного" значения. 
#если выберем самое популярное (моду), то мы просто сделаем его еще более популярным и увеличим СКО. 
#но и удалить мы их не можем, их слишком много.
#могу предположить, что стоит их выделить в отдельную группу 'no_data'

df_new[9].fillna('no_data', inplace = True)

In [286]:
df_new[11].fillna('no_data', inplace = True)

In [287]:
df_new[12].fillna('no_data', inplace = True)

In [288]:
df_new[17].fillna('no_data', inplace = True)

In [ ]:
#Кажется, что это пропуски MNAR?

In [290]:
# по 22 столбцу (что в итоге случилось с лошадью?) — 1 пропуск.
np.where(pd.isnull(df_new[22]))

#строка уже удалена, так что тут ничего не делаем.

(array([], dtype=int64),)

In [292]:
#вот итоговый df

df_new.head(40)

,0,1,4,9,11,12,17,22
0,2.0,1,66.0,2,4,4,5,2.0
1,1.0,1,88.0,1,4,2,2,3.0
2,2.0,1,40.0,1,3,1,1,1.0
3,1.0,9,164.0,2,4,4,no_data,2.0
4,2.0,1,104.0,2,no_data,no_data,no_data,2.0
5,2.0,1,48.0,1,3,2,3,1.0
6,1.0,1,48.0,1,3,3,5,1.0
7,1.0,1,60.0,1,4,2,4,2.0
8,2.0,1,80.0,1,4,4,5,3.0
9,2.0,9,90.0,1,3,1,no_data,1.0
